# **KSA Telecom's Customer Arabic Sentiment Analysis**

**Introduction**

The volume of Arabic posts on numerous social networks has increased dramatically, giving a rich supply of viewpoints on a wide range of topics. While opinion mining has sparked a lot of academic interest in the previous decade, it has primarily been done in English. Opinion mining from Arabic social media is behind, owing to difficulties in processing the complex Arabic natural language and a lack of tools and resources for labelling Arabic dataset and extracting Arabic feelings from the text. The goal of this project is to perform sentiment analysis on Arabic tweets using customer care tweets in the KSA. This data contains 10,000 tweets for a telecom company's customer care account on Twitter.

---

**Project Description**

The project was divided into four main phases:
  *   Preprocessing
  *   Labeling
  *   Feature Extraction
  *   Classification

---

**Source of Data:**

https://www.kaggle.com/datasets/mansourhussain/customer-care-tweets-ksa


## **Python Libraries**

In [ ]:
# Loading Libraries
import numpy as np
import pandas as pd

# Data Processing
import re
import string
import emoji
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Data Labelling
import httpx
from googletrans import Translator
from textblob import TextBlob

# Analysis and Visualisation
from tabulate import tabulate
import codecs
import matplotlib
from matplotlib import pyplot as plt

# Feature Extraction
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer

# Clasiffication
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix
import seaborn as sns

## **Data Uploading**

In [21]:
df = pd.read_csv('/File_Path/stc_care_tweets.csv', encoding="utf-8")

## **Preprocessing**

In [7]:
def pre_process(tweet):
    # Replace @username with empty string
    tweet = re.sub('@[^\s]+',' ', tweet)
    # Replace www.* or https?://* with empty string
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ', tweet)
    # Replace #word with with empty string
    tweet = re.sub('#([^\s]+)',' ', tweet)
    # Replace the numbers with empty string
    tweet = re.sub('\d+',' ', tweet)
    # Replace punctuations with empty string
    tweet = re.sub('[^\w\s]',' ', tweet)
    # Replace english alphabet letter with empty string
    tweet = re.sub('[a-zA-Z]',' ', tweet)
    return tweet

In [8]:
# Applying pre_process function for preprocessing
df["clean_text"] = df['Text'].apply(lambda x:pre_process(x))

In [9]:
df = df.applymap(lambda x:x.strip() if isinstance(x, str) else x)
df['clean_text'].replace('', np.nan, inplace=True)
df.dropna(subset=['clean_text'], inplace=True)

In [10]:
# Replace emoji with empty string
df['clean_text'] = df['clean_text'].apply(lambda s:emoji.replace_emoji(s, ''))
# Remove the daplicate raw
df = df.drop_duplicates()
# Tokenize the tweet text
df['clean_text'] = df['clean_text'].apply(nltk.word_tokenize)
# Remove the stop words
stopwords_list = stopwords.words('arabic')
df['clean_text'] = df['clean_text'].apply(lambda x:[item for item in x if item not in stopwords_list])
# lemmatize the tweet text
lemmatizer  = WordNetLemmatizer()
df['clean_text'] = df['clean_text'].apply(lambda x:[lemmatizer.lemmatize(word) for word in x])

## **Labilling**

In [11]:
timeout = httpx.Timeout(5) # 5 seconds timeout
translator = Translator(timeout=timeout)
df['clean_text'] = df['clean_text'].astype(str)
df['English'] = df['clean_text'].apply(translator.translate, src='ar', dest='en').apply(getattr, args=('text',))

In [ ]:
# TextBlob Data Lablling
from textblob import TextBlob
df['Sentiment'] = ''
for i,x in df.English.iteritems():
  label = TextBlob(x)
  df['Sentiment'][i] = label.sentiment.polarity
def polarity_to_label(x):
    if(x >= -1 and x < 0):
        return 'Negative'
    if(x == 0):
        return 'Neutral'
    if(x > 0 and x <= 1):
        return 'Positive'
df.Sentiment = df.Sentiment.apply(polarity_to_label)

## **Analysis and Visualisation**

In [ ]:
# Count total number of tweets
Tweets_Count = len(df)
# Count number of unique users
Unique_Users = len(df['Tweet Id'].unique())
dict_Tweets  = {'Name':['Tweets_Count', 'User_Count'], 'Count':[Tweets_Count, Unique_Users]}
head = ["Name", "Count"]
print(tabulate(dict_Tweets, headers=head, tablefmt="grid"))

In [ ]:
df['Sentiment'].value_counts()

In [ ]:
# People's feelings towards the Telecom companies
matplotlib.style.use('tableau-colorblind10')
df['Sentiment'].value_counts().plot.pie(autopct='%1.1f%%', wedgeprops={'linewidth':2.0, 'edgecolor': 'white'}, textprops={'fontsize':8})
plt.title("Customer Feelings Towards KSA Telecom", fontsize=10)
plt.axis('equal')
plt.show()

## **Features Extraction**

In [15]:
# TF-IDF features extraction
word_vectorizer = TfidfVectorizer(sublinear_tf=True,strip_accents='unicode', analyzer='word', ngram_range=(1, 1), max_features =1000)
unigramdataGet= word_vectorizer.fit_transform(df['clean_text'].astype('str'))
unigramdataGet = unigramdataGet.toarray()
vocab = word_vectorizer.get_feature_names_out()
features=pd.DataFrame(np.round(unigramdataGet, 1), columns=vocab)
features[features>0] = 1
features.head()
pro = preprocessing.LabelEncoder()
encpro = pro.fit_transform(df['Sentiment'])
df['label'] = encpro
y = df['Sentiment']
X = features

## **Classification**

In [16]:
# Spliting Dataset into 70% Training and 30% Testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=333)

In [ ]:
# RandomForest Algorithm
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
RF =clf.score(X_test, y_test)
print('Accuracy= {:.3f}'.format(clf.score(X_test, y_test)))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt="d")
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()